# การจองโรงแรมด้วย Middleware สำหรับสมาชิกพิเศษ

โน้ตบุ๊กนี้แสดงตัวอย่าง **Middleware แบบฟังก์ชัน** โดยใช้ Microsoft Agent Framework เราจะต่อยอดจากตัวอย่างการทำงานแบบมีเงื่อนไขโดยเพิ่มชั้น Middleware ที่ให้สิทธิพิเศษแก่สมาชิกที่มีความสำคัญ

## สิ่งที่คุณจะได้เรียนรู้:
1. **Middleware แบบฟังก์ชัน**: สกัดและปรับเปลี่ยนผลลัพธ์ของฟังก์ชัน
2. **การเข้าถึง Context**: อ่านและปรับเปลี่ยน `context.result` หลังการดำเนินการ
3. **การนำตรรกะทางธุรกิจไปใช้**: สิทธิพิเศษสำหรับสมาชิกที่มีความสำคัญ
4. **การเปลี่ยนแปลงผลลัพธ์**: ปรับเปลี่ยนผลลัพธ์ของฟังก์ชันตามสถานะของผู้ใช้
5. **การทำงานแบบเดิม แต่ผลลัพธ์ต่างกัน**: พฤติกรรมที่เปลี่ยนแปลงด้วย Middleware

## สถาปัตยกรรมการทำงานร่วมกับ Middleware:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## ความแตกต่างสำคัญจากการทำงานแบบมีเงื่อนไข:

**ไม่มี Middleware** (14-conditional-workflow.ipynb):
- ปารีสไม่มีห้องว่าง → ส่งไปยัง alternative_agent

**มี Middleware** (โน้ตบุ๊กนี้):
- ผู้ใช้ทั่วไป + ปารีส → ไม่มีห้องว่าง → ส่งไปยัง alternative_agent
- สมาชิกพิเศษ + ปารีส → 🌟 Middleware เข้ามาเปลี่ยนแปลง! → มีห้องว่าง → ส่งไปยัง booking_agent

## สิ่งที่ต้องเตรียม:
- ติดตั้ง Microsoft Agent Framework
- ความเข้าใจเกี่ยวกับการทำงานแบบมีเงื่อนไข (ดู 14-conditional-workflow.ipynb)
- GitHub token หรือ OpenAI API key
- ความเข้าใจพื้นฐานเกี่ยวกับรูปแบบ Middleware


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ขั้นตอนที่ 1: กำหนดโมเดล Pydantic สำหรับผลลัพธ์ที่มีโครงสร้าง

โมเดลเหล่านี้กำหนด **schema** ที่ตัวแทนจะส่งคืน เราได้เพิ่มฟิลด์ `priority_override` เพื่อใช้ติดตามเมื่อ middleware ปรับเปลี่ยนผลลัพธ์ความพร้อมใช้งาน


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ขั้นตอนที่ 2: กำหนดฐานข้อมูลสมาชิกที่มีสิทธิพิเศษ

สำหรับการสาธิตนี้ เราจะจำลองฐานข้อมูลสมาชิกที่มีสิทธิพิเศษ ในการใช้งานจริง ฐานข้อมูลนี้จะดึงข้อมูลจากฐานข้อมูลหรือ API จริง

**สมาชิกที่มีสิทธิพิเศษ:**
- `alice@example.com` - สมาชิก VIP
- `bob@example.com` - สมาชิกพรีเมียม  
- `priority_user` - บัญชีทดสอบ


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## ขั้นตอนที่ 3: สร้างเครื่องมือจองโรงแรม

เหมือนกับการทำงานแบบมีเงื่อนไข แต่ตอนนี้จะถูกสกัดกั้นโดย middleware!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ขั้นตอนที่ 4: 🌟 สร้าง Priority Check Middleware (ฟีเจอร์สำคัญ!)

นี่คือ **ฟังก์ชันหลัก** ของโน้ตบุ๊กนี้ Middleware:

1. **ดักจับ** การเรียกใช้งานฟังก์ชัน hotel_booking
2. **ดำเนินการ** ฟังก์ชันตามปกติโดยเรียก `next(context)`
3. **ตรวจสอบ** ผลลัพธ์ใน `context.result`
4. **ปรับเปลี่ยน** ผลลัพธ์หากผู้ใช้เป็น priority และไม่มีห้องว่าง
5. **ส่งคืน** ผลลัพธ์ที่ถูกปรับเปลี่ยนกลับไปยัง agent

**รูปแบบสำคัญ:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## ขั้นตอนที่ 5: กำหนดฟังก์ชันเงื่อนไขสำหรับการกำหนดเส้นทาง

ฟังก์ชันเงื่อนไขเหมือนกับในเวิร์กโฟลว์แบบมีเงื่อนไข - ใช้ตรวจสอบผลลัพธ์ที่มีโครงสร้างเพื่อกำหนดเส้นทาง.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## ขั้นตอนที่ 6: สร้าง Custom Display Executor

ตัวจัดการการทำงานเหมือนเดิม - แสดงผลลัพธ์สุดท้ายของเวิร์กโฟลว์


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## ขั้นตอนที่ 7: โหลดตัวแปรสภาพแวดล้อม

ตั้งค่าคลายแอนต์ LLM (GitHub Models หรือ OpenAI)


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## ขั้นตอนที่ 8: สร้าง AI Agents ด้วย Middleware

**ความแตกต่างสำคัญ:** เมื่อสร้าง availability_agent เราจะส่งพารามิเตอร์ `middleware`!

นี่คือวิธีที่เราใส่ priority_check_middleware เข้าไปในกระบวนการเรียกใช้งานฟังก์ชันของ agent.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ขั้นตอนที่ 9: สร้าง Workflow

โครงสร้าง Workflow เหมือนเดิม - การกำหนดเส้นทางตามเงื่อนไขโดยอิงจากความพร้อมใช้งาน


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## ขั้นตอนที่ 10: กรณีทดสอบ 1 - ผู้ใช้ทั่วไปในปารีส (ไม่มีการแทนที่)

ผู้ใช้ทั่วไปพยายามจองปารีส → ไม่มีห้องว่าง → ส่งต่อไปยัง alternative_agent


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## ขั้นตอนที่ 11: กรณีทดสอบที่ 2 - 🌟 ผู้ใช้ระดับ Priority ในปารีส (พร้อมการ Override!)

สมาชิกระดับ Priority พยายามจองห้องในปารีส → ไม่มีห้องว่างในตอนแรก → 🌟 Middleware เข้ามาแก้ไข! → ส่งต่อไปยัง booking_agent

**นี่คือการแสดงให้เห็นถึงพลังของ Middleware อย่างแท้จริง!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## ขั้นตอนที่ 12: กรณีทดสอบที่ 3 - ผู้ใช้ Priority ในสตอกโฮล์ม (มีห้องว่างอยู่แล้ว)

ผู้ใช้ Priority ลองจองที่สตอกโฮล์ม → มีห้องว่าง → ไม่จำเป็นต้องใช้การ override → ส่งต่อไปยัง booking_agent

นี่แสดงให้เห็นว่า middleware จะทำงานเฉพาะเมื่อจำเป็นเท่านั้น!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## ประเด็นสำคัญและแนวคิดเกี่ยวกับ Middleware

### ✅ สิ่งที่คุณได้เรียนรู้:

#### **1. รูปแบบ Middleware ที่ใช้ฟังก์ชัน**

Middleware สกัดกั้นการเรียกฟังก์ชันโดยใช้ฟังก์ชันแบบ async ง่าย ๆ:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. การเข้าถึง Context และการแก้ไขผลลัพธ์**

- `context.function` - เข้าถึงฟังก์ชันที่ถูกเรียก
- `context.arguments` - อ่านอาร์กิวเมนต์ของฟังก์ชัน
- `context.kwargs` - เข้าถึงพารามิเตอร์เพิ่มเติม
- `await next(context)` - เรียกใช้งานฟังก์ชัน
- `context.result` - อ่าน/แก้ไขผลลัพธ์ของฟังก์ชัน

#### **3. การนำตรรกะทางธุรกิจไปใช้**

Middleware ของเรานำเสนอสิทธิประโยชน์สำหรับสมาชิกที่มีลำดับความสำคัญ:
- **ผู้ใช้ทั่วไป**: ไม่มีการแก้ไข ใช้กระบวนการมาตรฐาน
- **ผู้ใช้ที่มีลำดับความสำคัญ**: แก้ไข "ไม่มีความพร้อมใช้งาน" → "พร้อมใช้งาน"
- **ตรรกะตามเงื่อนไข**: แก้ไขเฉพาะเมื่อจำเป็น

#### **4. กระบวนการเดิม ผลลัพธ์ที่แตกต่าง**

พลังของ Middleware:
- ✅ ไม่ต้องเปลี่ยนโครงสร้างของกระบวนการ
- ✅ ไม่ต้องเปลี่ยนฟังก์ชันเครื่องมือ
- ✅ ไม่ต้องเปลี่ยนตรรกะการกำหนดเส้นทางตามเงื่อนไข
- ✅ ใช้แค่ Middleware → พฤติกรรมที่แตกต่าง!

### 🚀 การใช้งานในโลกจริง:

1. **ฟีเจอร์ VIP/พรีเมียม**
   - แก้ไขข้อจำกัดการใช้งานสำหรับผู้ใช้พรีเมียม
   - ให้การเข้าถึงทรัพยากรแบบลำดับความสำคัญ
   - ปลดล็อกฟีเจอร์พรีเมียมแบบไดนามิก

2. **การทดสอบ A/B**
   - กำหนดเส้นทางผู้ใช้ไปยังการใช้งานที่แตกต่างกัน
   - ทดสอบฟีเจอร์ใหม่กับผู้ใช้เฉพาะกลุ่ม
   - เปิดตัวฟีเจอร์ใหม่แบบค่อยเป็นค่อยไป

3. **ความปลอดภัยและการปฏิบัติตามกฎระเบียบ**
   - ตรวจสอบการเรียกฟังก์ชัน
   - บล็อกการดำเนินการที่ละเอียดอ่อน
   - บังคับใช้กฎทางธุรกิจ

4. **การปรับปรุงประสิทธิภาพ**
   - แคชผลลัพธ์สำหรับผู้ใช้เฉพาะกลุ่ม
   - ข้ามการดำเนินการที่มีค่าใช้จ่ายสูงเมื่อเป็นไปได้
   - การจัดสรรทรัพยากรแบบไดนามิก

5. **การจัดการข้อผิดพลาดและการลองใหม่**
   - จับและจัดการข้อผิดพลาดอย่างราบรื่น
   - ใช้ตรรกะการลองใหม่
   - ใช้การดำเนินการสำรองเมื่อจำเป็น

6. **การบันทึกและการตรวจสอบ**
   - ติดตามเวลาการดำเนินการของฟังก์ชัน
   - บันทึกพารามิเตอร์และผลลัพธ์
   - ตรวจสอบรูปแบบการใช้งาน

### 🔑 ความแตกต่างสำคัญจาก Decorators:

| คุณสมบัติ | Decorator | Middleware |
|-----------|-----------|------------|
| **ขอบเขต** | ฟังก์ชันเดียว | ฟังก์ชันทั้งหมดใน agent |
| **ความยืดหยุ่น** | กำหนดไว้ตอนนิยาม | เปลี่ยนแปลงได้ตอน runtime |
| **Context** | จำกัด | Context ของ agent ทั้งหมด |
| **การประกอบ** | Decorators หลายตัว | Middleware pipeline |
| **การรับรู้ agent** | ไม่ | ใช่ (เข้าถึงสถานะของ agent) |

### 📚 เมื่อใดควรใช้ Middleware:

✅ **ใช้ Middleware เมื่อ:**
- คุณต้องการแก้ไขพฤติกรรมตามสถานะผู้ใช้/เซสชัน
- คุณต้องการใช้ตรรกะกับหลายฟังก์ชัน
- คุณต้องการเข้าถึง Context ระดับ agent
- คุณกำลังนำเสนอข้อกังวลที่ครอบคลุม (การบันทึก, การตรวจสอบสิทธิ์ ฯลฯ)

❌ **อย่าใช้ Middleware เมื่อ:**
- การตรวจสอบความถูกต้องของข้อมูลเข้าแบบง่าย ๆ (ใช้ Pydantic)
- ตรรกะเฉพาะฟังก์ชัน (เก็บไว้ในฟังก์ชัน)
- การแก้ไขครั้งเดียว (แค่เปลี่ยนฟังก์ชัน)

### 🎓 รูปแบบขั้นสูง:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 แนวคิดที่เกี่ยวข้อง:

- **Agent Middleware**: สกัดกั้นการเรียก agent.run()
- **Function Middleware**: สกัดกั้นการเรียกฟังก์ชันเครื่องมือ (สิ่งที่เราใช้!)
- **Middleware Pipeline**: ลำดับของ Middleware ที่ดำเนินการตามลำดับ
- **Context Propagation**: ส่งผ่านสถานะผ่าน chain ของ Middleware



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์ที่มีความเชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
